In [1]:
import requests
from requests.auth import HTTPProxyAuth
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import socks
import socket
import sqlite3
from stem import Signal
from stem.control import Controller 
import numpy as np
import math
import json
import spacy
import urllib.parse
import math
nlp = spacy.load('en')

In [50]:
#Create SQLite
# conn = sqlite3.connect('songlyrics.db')
# df = pd.read_sql_query("select * from songlyrics_url where status=1;",conn)
conn = sqlite3.connect('spotify.db')




header = {"Authorization":"Bearer "+auth_token}  

In [4]:
soup = BeautifulSoup(df['fullhtml'][3],"lxml")
df['fullhtml'][10]
lyrics = soup.find('p',attrs={'id':'songLyricsDiv'})
lyric = lyrics.get_text()

In [93]:
lyric_clean = re.sub(r'\n[\(]?Chorus[:]?[\)]?[ ]?\n',"",lyric)
lyric_final = lyric_clean.replace("\n\n","\n").replace("\n",".\n").replace(" .",".")

In [94]:
doc = nlp(lyric_final)

In [91]:
for s in doc.sents:
    print(s)
    print('-----------')

Let now the weak say I have strength.

-----------
By the Spirit of pow'r.

-----------
That raised Christ from the dead.

-----------
Let now the poor stand and confess.
That my portion in Him.

-----------
And I'm more then blessed.

-----------
Let now our hearts burn with a flame.

-----------
A fire consuming all for Your Son's holy name.

-----------
And with the heavens we declare.

-----------
You are our King.

-----------
We love You, Lord.

-----------
We Worship You.

-----------
You are our God.

-----------
You alone are good.

-----------
Let now Your church shine as your bride.

-----------
That You saw in Your heart.

-----------
As You offered up Your life.

-----------
Let now the lost be welcome home.

-----------
By the saved and redeemed.

-----------
Those adopted as Your own.

-----------
Let now our hearts burn with a flame.

-----------
A fire consuming all for Your Son's holy name.

-----------
And with the heavens we declare.

-----------
You are our King.



In [95]:
url = 'https://language.googleapis.com/v1beta2/documents:analyzeSentiment?key='+apikey

data = {"document": {"type":"PLAIN_TEXT","content":lyric_final},"encodingType":"UTF8"}

response = requests.post(url,json=data)
json.loads(response.text)


{'documentSentiment': {'magnitude': 17.1, 'score': 0.3},
 'language': 'en',
 'sentences': [{'sentiment': {'magnitude': 0.3, 'score': -0.3},
   'text': {'beginOffset': 0,
    'content': 'Let now the weak say I have strength.'}},
  {'sentiment': {'magnitude': 0.3, 'score': 0.3},
   'text': {'beginOffset': 38, 'content': "By the Spirit of pow'r."}},
  {'sentiment': {'magnitude': 0.1, 'score': 0.1},
   'text': {'beginOffset': 62,
    'content': 'That raised Christ from the dead.'}},
  {'sentiment': {'magnitude': 0.7, 'score': -0.7},
   'text': {'beginOffset': 96,
    'content': 'Let now the poor stand and confess.'}},
  {'sentiment': {'magnitude': 0, 'score': 0},
   'text': {'beginOffset': 132, 'content': 'That my portion in Him.'}},
  {'sentiment': {'magnitude': 0.8, 'score': 0.8},
   'text': {'beginOffset': 156, 'content': "And I'm more then blessed."}},
  {'sentiment': {'magnitude': 0.2, 'score': 0.2},
   'text': {'beginOffset': 183,
    'content': 'Let now our hearts burn with a flame.

In [96]:
json.loads(response.text)

{'documentSentiment': {'magnitude': 17.1, 'score': 0.3},
 'language': 'en',
 'sentences': [{'sentiment': {'magnitude': 0.3, 'score': -0.3},
   'text': {'beginOffset': 0,
    'content': 'Let now the weak say I have strength.'}},
  {'sentiment': {'magnitude': 0.3, 'score': 0.3},
   'text': {'beginOffset': 38, 'content': "By the Spirit of pow'r."}},
  {'sentiment': {'magnitude': 0.1, 'score': 0.1},
   'text': {'beginOffset': 62,
    'content': 'That raised Christ from the dead.'}},
  {'sentiment': {'magnitude': 0.7, 'score': -0.7},
   'text': {'beginOffset': 96,
    'content': 'Let now the poor stand and confess.'}},
  {'sentiment': {'magnitude': 0, 'score': 0},
   'text': {'beginOffset': 132, 'content': 'That my portion in Him.'}},
  {'sentiment': {'magnitude': 0.8, 'score': 0.8},
   'text': {'beginOffset': 156, 'content': "And I'm more then blessed."}},
  {'sentiment': {'magnitude': 0.2, 'score': 0.2},
   'text': {'beginOffset': 183,
    'content': 'Let now our hearts burn with a flame.

Spotify API
https://developer.spotify.com/web-api/console/get-search-item/

In [134]:
#Get Artist
search = "Claire Cloninger".replace(" ","%20")              

url = 'https://api.spotify.com/v1/search?q='+search+'&type=artist'
print(url)
artist = requests.get(url,headers=header).json()
# print(artist)
popularity = 0 
for i in artist['artists']['items']:
    if i['popularity'] > popularity:
        popularity = i['popularity']
        artist = [i['id'],i['name'],i['genres'],i['popularity'],i['followers']['total']]
print(artist)    
#     print(i['genres'])
#     print(i['popularity'])
#     print(i['followers']['total'])
#     print(i['name'])
#     print(i['id'])
#     print('--------------')

https://api.spotify.com/v1/search?q=Claire%20Cloninger&type=artist
{'artists': {'previous': None, 'items': [], 'href': 'https://api.spotify.com/v1/search?query=Claire+Cloninger&type=artist&market=SG&offset=0&limit=20', 'next': None, 'limit': 20, 'offset': 0, 'total': 0}}


In [124]:
# Get Albums
columns = ['album_name','album_id','album_type']
df_albums = pd.DataFrame(columns=columns)

url = 'https://api.spotify.com/v1/artists/'+artist[0]+"/albums"
# url = 'https://api.spotify.com/v1/artists/'+artist['artists']['items'][0]['id']+"/albums"

# print(url)

iter = 0
albums = requests.get(url,headers=header).json()
while True:
    for a in albums['items']:
        row = [a['name'],a['id'],a['album_type']]
        df_albums.loc[len(df_albums)] = row  
    if (albums['next'] == None)|(iter>10) : 
        break
    else: 
        print(albums['next'])
        albums = requests.get(albums['next'],headers=header).json()
        iter = iter + 1

print(len(df_albums))
print(len(df_albums['album_id'].unique()))

5
5


In [125]:
query_album_id = df_albums['album_id']

list_album_df = []
start = 0 
end = 0 
max_per_call = 20
iteration = math.ceil(len(query_album_id)/max_per_call)

for i in range(iteration):
    start = i*max_per_call
    end = start+max_per_call
    query_multiple = ','.join(query_album_id[start:end])
    
    album_url = "https://api.spotify.com/v1/albums/?ids="+query_multiple

    album_detail = requests.get(album_url,headers=header).json()
        
    adf = pd.DataFrame(album_detail['albums']).loc[:,['popularity','release_date','name','id','type']]
    
    list_album_df.append(adf)

dfalbums = pd.concat(list_album_df)
dfalbums=dfalbums.rename(columns = {'id':'album_id'})
dfalbums=dfalbums.rename(columns = {'name':'album_name'})
dfalbums=dfalbums.rename(columns = {'popularity':'album_popularity'})

print(len(dfalbums))
print(len(dfalbums['album_id'].unique()))
dfalbums.head(3)

5
5


,album_popularity,release_date,album_name,album_id,type
0,33,2015-05-15,"Kids Praise! 2 ""a Joy-Fulliest Noise!""",1cUEZ74NumnG8gdgKzCRFH,album
1,30,2015-05-15,"Kids Praise! 4 ""Singsational Servants!""",0q1KqZh0yvYJxTwFavblIH,album
2,32,2014-05-20,"Kids Praise! 3 ""Funtastic Family!""",4WzuTuBByMhBxm90SmNGG2,album


In [126]:
# Get Tracks ID in Albums
# Loop all albums

columns = ['album_name','album_id','release_date','album_popularity','album_type','track_id','track_name','track_artist']
dftracks = pd.DataFrame(columns=columns)

for key,row in dfalbums.iterrows():
    pause = randint(0,1)
    time.sleep(pause)
#     print(row['album_id'])
    url = 'https://api.spotify.com/v1/albums/'+row['album_id']+"/tracks"
#     print(url)
    tracks = requests.get(url,headers=header).json()
    iter = 0 
    while True:
        
        for t in tracks['items']:
            insert = [row['album_name'],row['album_id'],row['release_date'],row['album_popularity'],row['type'],t['id'],t['name'],t['artists'][0]['name']]
            dftracks.loc[len(dftracks)] = insert
            
        if (tracks['next'] == None)|(iter>5) : 
            break
        else: 
            tracks = requests.get(tracks['next'],headers=header).json()
            iter = iter + 1


print(len(dftracks[dftracks['track_artist']=='Twila Paris']))
print(len(dftracks))


0
103


In [127]:
query_list = dftracks['track_id']
len(query_list)

list_df = []
start = 0 
end = 0 
max_per_call = 50
iteration = math.ceil(len(query_list)/max_per_call)

for i in range(iteration):
    start = i*max_per_call
    end = start+max_per_call
    query_multiple = ','.join(query_list[start:end])
#     print(query_multiple)
    
    feature_url = 'https://api.spotify.com/v1/audio-features?ids='+query_multiple
    track_url = 'https://api.spotify.com/v1/tracks?ids='+query_multiple
    if i%10 == 0 : print(i)
#     print(query_multiple)
    track_features = requests.get(feature_url,headers=header).json()
    track_details = requests.get(track_url,headers=header).json()
    
    tfaf = [i for i in track_features['audio_features'] if i!=None]
    tdt = [i for i in track_details['tracks'] if i!=None]
    
    df = pd.DataFrame(tfaf).loc[:,['acousticness','danceability','duration_ms','energy','id','instrumentalness','key','liveness','loudness','mode','speechiness','tempo','time_signature','valence']]
    tf = pd.DataFrame(tdt).loc[:,['id','name','artists']]

    consolidated_features = tf.merge(df,how='inner')
    list_df.append(consolidated_features)



0


In [128]:
final_track_df = pd.concat(list_df)

def artist(x):
#     artist_list[artist_list['Genre']=='Band'].iloc[20,0].split('\xa0')[0]
    return x['artists'][0]['name']
    
final_track_df['singer'] = final_track_df.apply(lambda x:artist(x),axis=1)
del final_track_df['artists']
final_track_df.head(3)


# for tf in track_features['audio_features']:
#     print(tf)
# for td in track_details['tracks']:
#     print(td['id'],td['name'],td['popularity'])

,id,name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,singer
0,18DPR68iAGj5M9achPtOzM,Scene 1 - It's a Giant Blue Book!,0.526,0.684,52445,0.171,0.0,7,0.381,-25.359,1,0.9010,126.743,3.0,0.742,Psalty
1,3vOFGoz0GbdsmGm9p9VlRW,Matthew 16:24 (If Any Man),0.456,0.746,151362,0.436,0.0,3,0.461,-13.571,1,0.0729,126.989,4.0,0.852,Psalty
2,19UioFfikx5FLjnKUgcYnw,Scene 2 - Those Statues Are Moving,0.696,0.719,99240,0.214,0.0,4,0.950,-22.905,0,0.9480,121.005,3.0,0.494,Psalty


In [129]:
print(len(final_track_df['id'].unique()))
print(len(final_track_df))

print(len(dftracks['track_id'].unique()))
print(len(dftracks))

103
103
103
103


In [130]:
final_df = final_track_df.merge(dftracks,left_on='id',right_on='track_id',how='inner')

In [131]:
print(len(final_df))
final_df.head(3)

103


,id,name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,...,valence,singer,album_name,album_id,release_date,album_popularity,album_type,track_id,track_name,track_artist
0,18DPR68iAGj5M9achPtOzM,Scene 1 - It's a Giant Blue Book!,0.526,0.684,52445,0.171,0.0,7,0.381,-25.359,...,0.742,Psalty,"Kids Praise! 2 ""a Joy-Fulliest Noise!""",1cUEZ74NumnG8gdgKzCRFH,2015-05-15,33.0,album,18DPR68iAGj5M9achPtOzM,Scene 1 - It's a Giant Blue Book!,Psalty
1,3vOFGoz0GbdsmGm9p9VlRW,Matthew 16:24 (If Any Man),0.456,0.746,151362,0.436,0.0,3,0.461,-13.571,...,0.852,Psalty,"Kids Praise! 2 ""a Joy-Fulliest Noise!""",1cUEZ74NumnG8gdgKzCRFH,2015-05-15,33.0,album,3vOFGoz0GbdsmGm9p9VlRW,Matthew 16:24 (If Any Man),Psalty
2,19UioFfikx5FLjnKUgcYnw,Scene 2 - Those Statues Are Moving,0.696,0.719,99240,0.214,0.0,4,0.950,-22.905,...,0.494,Psalty,"Kids Praise! 2 ""a Joy-Fulliest Noise!""",1cUEZ74NumnG8gdgKzCRFH,2015-05-15,33.0,album,19UioFfikx5FLjnKUgcYnw,Scene 2 - Those Statues Are Moving,Psalty


In [132]:
final_df.to_sql('tracks_analysis',conn,if_exists='append')

In [136]:
export = pd.read_sql_query("select * from tracks_analysis;",conn)
export.to_csv('export.csv')

In [ ]:
response.request.headers

Musixmatch
https://developer.musixmatch.com/